In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from datetime import datetime
import json

In [2]:
# Load the data
train_data = pd.read_csv("data_final_project/KuaiRec 2.0/data/big_matrix.csv")
test_data = pd.read_csv("data_final_project/KuaiRec 2.0/data/small_matrix.csv")
kuairec_caption_category = pd.read_csv(
    "data_final_project/KuaiRec 2.0/data/kuairec_caption_category.csv",
    engine="python", sep=",", quotechar='"', on_bad_lines='skip'
)
item_categories = pd.read_csv("data_final_project/KuaiRec 2.0/data/item_categories.csv")
item_daily_features = pd.read_csv("data_final_project/KuaiRec 2.0/data/item_daily_features.csv")
social_network = pd.read_csv("data_final_project/KuaiRec 2.0/data/social_network.csv")
user_features = pd.read_csv("data_final_project/KuaiRec 2.0/data/user_features.csv")


In [3]:
display(train_data.head(5000))

,user_id,video_id,play_duration,video_duration,time,date,timestamp,watch_ratio
0,0,3649,13838,10867,2020-07-05 00:08:23.438,20200705,1.593879e+09,1.273397
1,0,9598,13665,10984,2020-07-05 00:13:41.297,20200705,1.593879e+09,1.244082
2,0,5262,851,7908,2020-07-05 00:16:06.687,20200705,1.593879e+09,0.107613
3,0,1963,862,9590,2020-07-05 00:20:26.792,20200705,1.593880e+09,0.089885
4,0,8234,858,11000,2020-07-05 00:43:05.128,20200705,1.593881e+09,0.078000
...,...,...,...,...,...,...,...,...
4995,2,3382,9941,11055,2020-09-03 04:47:22.596,20200903,1.599080e+09,0.899231
4996,2,2935,3425,12095,2020-09-03 04:48:19.45,20200903,1.599080e+09,0.283175
4997,2,6309,3726,6067,2020-09-03 04:49:44.539,20200903,1.599080e+09,0.614142
4998,2,4735,5309,9067,2020-09-03 04:51:08.016,20200903,1.599080e+09,0.585530


In [ ]:
# Create a positive interaction indicator based on watch time
# If the user watches at least 50% of the video, it is considered positive feedback
train_data['positive_interaction'] = (train_data['watch_ratio'] >= 0.5).astype(int)
test_data['positive_interaction'] = (test_data['watch_ratio'] >= 0.5).astype(int)

In [5]:
# Convert 'friend_list' from string to list
social_network['friend_list'] = social_network['friend_list'].apply(
    lambda x: json.loads(x.replace("'", '"')) if isinstance(x, str) else x
)

social_network['num_friends'] = social_network['friend_list'].apply(len)

In [9]:
test_users = test_data['user_id'].unique()
test_videos = train_data['video_id'].unique()

In [10]:
# # Example of submission format (top 10 videos for each user)
# sample_submission = pd.DataFrame({
#     'user_id': np.repeat(test_users, 10),
#     'video_id': [0] * (len(test_users) * 10),  # Will be replaced by actual predictions
#     'rank': np.tile(range(1, 11), len(test_users))
# })
# sample_submission.to_csv('sample_submission.csv', index=False)

# print("Preprocessing finished. Files saved: interactions_train_train.csv, interactions_train_test.csv, sample_submission.csv")

In [11]:
num_users = train_data['user_id'].nunique()
num_videos = train_data['video_id'].nunique()

print(f"Number of unique users in train data: {num_users}")
print(f"Number of unique videos in train data: {num_videos}")

Number of unique users in train data: 7176
Number of unique videos in train data: 10728


In [12]:
# # Create a user-item interaction matrix
# def create_interaction_matrix(data, user_col='user_id', item_col='video_id', rating_col='positive_interaction'):
#     """
#     Creates an interaction matrix between users and items.
#     """
#     interactions_train = data.groupby([user_col, item_col])[rating_col].sum().unstack().fillna(0)
#     return interactions_train

# # Create the matrix for the training set
# train_matrix = create_interaction_matrix(train_data)
# print(f"Training interaction matrix dimensions: {train_matrix.shape}")
# print(f"Matrix density: {train_matrix.count().sum() / (train_matrix.shape[0] * train_matrix.shape[1]):.4f}")

# # Save the interaction matrix
# train_matrix.to_pickle('data/train_interaction_matrix.pkl')